In [3]:
import os
if (os.path.isdir('D:\\Documents\\DMProject\\BNP\\')):
    os.chdir('D:\\Documents\\DMProject\\BNP\\')
if (os.path.isdir('C:\\Users\\Chao\\Documents\\DMProject\\BNP\\')):
    os.chdir('C:\\Users\\Chao\\Documents\\DMProject\\BNP\\')
os.getcwd()
os.listdir()

['input', 'lr_dispression_sfm.csv', 'lr_dispression_sfm_0.25.csv']

In [4]:
import numpy as np
import pandas as pd

In [5]:
def dispersion_col(df_train, df_test, col_name):
    if (df_train[col_name].dtype != 'O'):
#         print(col_name + "is number")
        return
    g = df_train.groupby(col_name)
    if len(g.size()) > 200:
#         print(col_name + "group to big")
        return
    
    for name, group in g:
        new_col_name = col_name + name
        df_train[new_col_name] = 0
        df_test[new_col_name] = 0
    
    for name, group in g:
        new_col_name = col_name + name
        df_train.loc[df_train[col_name] == name, new_col_name] = 1
        df_test.loc[df_test[col_name] == name, new_col_name] = 1
        
    
    df_train.drop([col_name], axis=1)
    df_test.drop([col_name], axis=1)

In [6]:
train = pd.read_csv('input\\train.csv')
test = pd.read_csv('input\\test.csv')
target = train['target']
id_test = test['ID'].values

In [7]:
len(target), sum(target)

(114321, 87021)

In [8]:
train = train.drop(['ID', 'target'], axis=1)
test = test.drop(['ID'], axis=1)
column_names = train.columns

In [9]:
# dispersion_col(train, 'v3')

for col_name in column_names:
    dispersion_col(train, test, col_name)

In [10]:
# train.columns.values

In [11]:
train = train.drop(['v3', 'v22', 'v24', 'v30', 'v31', 
                    'v47', 'v52', 'v56', 'v66', 'v71', 
                    'v74', 'v75', 'v79', 'v91', 
                    'v107', 'v110', 'v112', 'v113', 'v125'], axis=1)
test = test.drop(['v3', 'v22', 'v24', 'v30', 'v31', 
                    'v47', 'v52', 'v56', 'v66', 'v71', 
                    'v74', 'v75', 'v79', 'v91', 
                    'v107', 'v110', 'v112', 'v113', 'v125'], axis=1)


In [12]:
future = train.columns.values

In [13]:
from sklearn.preprocessing import Imputer
imp = Imputer(missing_values='NaN', strategy='mean', axis=0)
imp.fit(train)

Imputer(axis=0, copy=True, missing_values='NaN', strategy='mean', verbose=0)

In [14]:
train_imputed = imp.transform(train)
test_imputed = imp.transform(test)

In [15]:
from sklearn.linear_model import LogisticRegression
from sklearn import cross_validation
from sklearn import linear_model
from sklearn.feature_selection import SelectFromModel

In [16]:
alg    = LogisticRegression(random_state=1)

In [17]:
alg.fit(train_imputed, target)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=1, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [24]:
from sklearn.linear_model import LogisticRegression
from sklearn import cross_validation

# predictors = ["Pclass", "is_male", "is_female", "Age", "SibSp", "Parch", "Fare", "EmbS", "EmbC", "EmbQ"]

alg    = LogisticRegression(random_state=1)
scores = cross_validation.cross_val_score(
    alg,
    train_imputed,
    target,
    cv=3
)

print(scores.mean())

0.774284689602


In [26]:
alg.fit(train_imputed, target)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=1, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [27]:
y_pred = alg.predict_proba(test_imputed)
#print y_pred

pd.DataFrame({"ID": id_test, "PredictedProb": y_pred[:,1]}).to_csv('lr_dispression_cv.csv',index=False)

In [58]:
sfm = SelectFromModel(alg, threshold=0.25)
sfm.fit(train_imputed, target)

SelectFromModel(estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=1, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
        prefit=False, threshold=0.25)

In [59]:
n_features = sfm.transform(train_imputed).shape[1]

In [60]:
train_sfm = sfm.transform(train_imputed)
test_sfm = sfm.transform(test_imputed)

In [61]:
train_imputed.shape, train_sfm.shape

((114321, 476), (114321, 138))

In [62]:

alg.fit(train_sfm, target)
# scores = cross_validation.cross_val_score(
#     alg,
#     train_imputed[future],
#     target,
#     cv=3.
# )

# print(scores.mean())

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=1, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [63]:
y_pred = alg.predict_proba(test_sfm)
len(y_pred)

114393

In [64]:
y_value = alg.predict(test_sfm)

In [65]:
#print y_pred

pd.DataFrame({"ID": id_test, "PredictedProb": y_pred[:,1]}).to_csv('lr_dispression_sfm_0.25.csv',index=False)

In [66]:
type(y_pred[:,1]), type(y_value)

(numpy.ndarray, numpy.ndarray)

In [ ]:
id_test_2